In [2]:
# Read input data
with open("input.txt") as f:
    text = f.read()


In [3]:
print("length of the dataset in characters", len(text))


length of the dataset in characters 1115394


In [4]:
# First 1000 characters of the input data
display(text[:1000])


"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [5]:
# all unique characters in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# Create simple character tokenizer
stoi = {c: i for i, c in enumerate(chars)}
itos = {i: c for i, c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])
print(encode("Hello world!"))
print(decode(encode("Hello world!")))


[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
Hello world!


In [7]:
# Import torch and encode input data to tensor
import torch

data = torch.tensor(encode(text), dtype=torch.long, device=None)
print(data.shape, data.dtype)
print(data[:100])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
# Split encoded input into train and validation set
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


In [15]:
torch.manual_seed(42)
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

# Context visualization
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target is: {target}")


torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
When input is [57] the target is: 1
When input is [57, 1] the target is: 46
When input is [57, 1, 46] the target is: 47
When input is [57, 1, 46, 47] the target is: 57
When input is [57, 1, 46, 47, 57] the target is: 1
When input is [57, 1, 46, 47, 57, 1] the target is: 50
When input is [57, 1, 46, 47, 57, 1, 50] the target is: 53
When input is [57, 1, 46, 47, 57, 1, 50, 53] the target is: 60
When input is [1] the target is: 58
When input is [1, 58] the target is: 46
When input is [1, 58, 46] the target is: 43
When input is [1, 58, 46, 43] the target is: 56
When input is [1, 58, 46, 43, 56] the target is: 43
When inp

Bigram model

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=-1)  # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8865, grad_fn=<NllLossBackward0>)

o$,q&IWqW&xtCjaB?ij&bYRGkF?b; f ,CbwhtERCIfuWr,DzJERjhLlVaF&EjffPHDFcNoGIG'&$qXisWTkJPw
 ,b Xgx?D3sj


Train loop

In [12]:
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

for steps in range(2000):
    xb, yb = get_batch('train')

    logits, loss =  m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.1995608806610107


Generate output from model

In [13]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


D,DJV!g: IwVzIs& Zus: VdircAse ker,heetigsmavwZisby
abdyZGimgz$V,-
criXvQFqn'deQyrolicQ!Az-IAn,pw,Dp


The mathematical trick in self attention

In [14]:
torch.manual_seed(42)
B,T,C = 4,8,2
x = torch.rand(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [17]:
# Version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, dim=0)

In [28]:
# Version 2
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x

In [29]:
# Version 3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x

Self attention for one head

In [31]:
torch.manual_seed(42)
B,T,C = 4,8,32
x = torch.rand(B,T,C)

# single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B,T,16)
q = query(x) # (B,T,16)
wei = q @ k.transpose(-2, -1)  # (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

v = value(x)
out = wei @ v
out

tensor([[[ 2.6378e-01,  1.4320e-01,  1.0612e+00,  2.4182e-01, -3.6273e-01,
           5.3863e-03, -7.1514e-01, -5.8126e-01, -3.9213e-01,  2.7544e-01,
          -6.5308e-01, -3.8467e-01,  3.3778e-01, -5.4164e-01, -2.6631e-01,
           4.0298e-01],
         [ 1.4327e-01,  7.9573e-02,  1.0511e+00,  2.3275e-01, -2.7872e-01,
          -5.7517e-02, -5.7715e-01, -6.2039e-01, -4.6231e-01,  2.9281e-01,
          -8.4379e-01, -2.9442e-01,  4.0584e-01, -5.5960e-01, -1.9425e-01,
           4.8614e-01],
         [ 1.7828e-01,  4.6072e-02,  7.9989e-01,  1.3813e-01, -3.2865e-01,
          -2.9892e-02, -3.8560e-01, -5.4209e-01, -4.2164e-01,  2.7821e-01,
          -7.0592e-01, -3.4510e-01,  3.7394e-01, -4.5291e-01, -6.1384e-02,
           4.5229e-01],
         [ 1.5716e-01,  3.0422e-02,  6.9421e-01,  1.4530e-01, -2.8683e-01,
          -2.3814e-02, -3.2727e-01, -4.4452e-01, -3.3291e-01,  2.4523e-01,
          -6.7720e-01, -3.5894e-01,  3.4341e-01, -4.0172e-01, -6.1276e-02,
           4.0395e-01],
    